In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('/content/faulty_crop_datatset.csv')

data.head()
data.drop(columns=['Link'], inplace=True)

# Encode categorical features: District_Name, Soil_color
data = pd.get_dummies(data, columns=['District_Name', 'Soil_color'])

# Encode target labels
le_crop = LabelEncoder()
le_fert = LabelEncoder()
data['Crop'] = le_crop.fit_transform(data['Crop'])
data['Fertilizer'] = le_fert.fit_transform(data['Fertilizer'])

# Features and labels
X = data.drop(columns=['Crop', 'Fertilizer'])
y_crop = data['Crop']
y_fert = data['Fertilizer']

# Split into train/test
X_train, X_test, y_crop_train, y_crop_test, y_fert_train, y_fert_test = train_test_split(
    X, y_crop, y_fert, test_size=0.25, random_state=42
)

# Metric calculation function
def evaluate(model, X_train, X_test, y_crop_train, y_crop_test, y_fert_train, y_fert_test):
    # Crop prediction
    model.fit(X_train, y_crop_train)
    y_crop_pred = model.predict(X_test)

    # Fertilizer prediction
    model.fit(X_train, y_fert_train)
    y_fert_pred = model.predict(X_test)

    # Metrics for crop
    acc_crop = accuracy_score(y_crop_test, y_crop_pred)
    prec_crop = precision_score(y_crop_test, y_crop_pred, average='weighted')
    rec_crop = recall_score(y_crop_test, y_crop_pred, average='weighted')
    f1_crop = f1_score(y_crop_test, y_crop_pred, average='weighted')

    # Metrics for fertilizer
    acc_fert = accuracy_score(y_fert_test, y_fert_pred)
    prec_fert = precision_score(y_fert_test, y_fert_pred, average='weighted')
    rec_fert = recall_score(y_fert_test, y_fert_pred, average='weighted')
    f1_fert = f1_score(y_fert_test, y_fert_pred, average='weighted')

    # Correct overall accuracy (both correct)
    acc_overall = np.mean((y_crop_pred == y_crop_test) & (y_fert_pred == y_fert_test))

    return {
        'accuracy_crop': acc_crop,
        'accuracy_fert': acc_fert,
        'precision_crop': prec_crop,
        'precision_fert': prec_fert,
        'recall_crop': rec_crop,
        'recall_fert': rec_fert,
        'f1_crop': f1_crop,
        'f1_fert': f1_fert ,
        'accuracy_overall': acc_overall
    }

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(),
    "SVM (RBF)": SVC(kernel='rbf'),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

# Evaluate and store results
results = pd.DataFrame()
for name, model in models.items():
    m = evaluate(model, X_train, X_test, y_crop_train, y_crop_test, y_fert_train, y_fert_test)
    m['Model'] = name
    results = pd.concat([results, pd.DataFrame([m])], ignore_index=True)

# Final table
results.set_index('Model', inplace=True)
results

,accuracy_crop,accuracy_fert,precision_crop,precision_fert,recall_crop,recall_fert,f1_crop,f1_fert,accuracy_overall
Model,,,,,,,,,
Logistic Regression,0.807795,0.356953,0.802182,0.335060,0.807795,0.356953,0.801975,0.319627,0.303809
KNN,0.948627,0.534987,0.949371,0.536901,0.948627,0.534987,0.948546,0.532272,0.511072
SVM (RBF),0.601417,0.301151,0.571780,0.090692,0.601417,0.301151,0.570625,0.139403,0.202834
XGBoost,1.000000,0.947741,1.000000,0.952082,1.000000,0.947741,1.000000,0.947965,0.947741
